In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [3]:
load_dotenv(override=True)

python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10


True

In [5]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("muhammadhassan.ai@muhammadhassan.tech")  # Change to your verified sender
    to_email = To("muhammad.hassan.519570@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-5.2-chat-latest"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-5.2-chat-latest"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-5.2-chat-latest"
)

In [8]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplifying SOC 2 Compliance Without the Manual Work

Hi {{First Name}},

I’m reaching out because many growing companies I speak with are struggling to keep SOC 2 compliance on track while scaling their teams and infrastructure.

ComplAI is a SOC 2 compliance platform that automates evidence collection, control monitoring, and audit preparation using AI—so your team spends less time chasing screenshots and spreadsheets, and more time building product. Our customers typically cut audit prep time by 50–70% and go into audits with far fewer surprises.

If SOC 2 is on your roadmap (or already a recurring headache), I’d be happy to share how companies like yours are using ComplAI to:
- Maintain continuous SOC 2 readiness  
- Reduce reliance on consultants  
- Streamline communication with auditors  

Would you be open to a quick 15-minute conversation next week to see if this is relevant for {{Company Name}}?

Best regards,  
{{Your Name}}  
{{Title}} | ComplAI  
{{Email}} | {{Web

In [9]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
outputs = [result.final_output for result in results]
for i, output in enumerate(outputs, start=1):
    print(f"\n\n--- Sales Agent {i} Output ---\n")
    print(output)




--- Sales Agent 1 Output ---

**Subject:** Simplify SOC 2 Compliance—Without the Headaches

Hi {{First Name}},

I’m reaching out because many growing teams tell us that SOC 2 compliance is time‑consuming, expensive, and difficult to manage alongside day‑to‑day priorities.

ComplAI helps companies prepare for and maintain SOC 2 compliance with far less effort. Our AI‑powered platform automates evidence collection, maps controls to your systems, and keeps you audit‑ready year‑round—so you’re not scrambling when an audit or customer security review comes up.

Teams typically use ComplAI to:
- Cut SOC 2 prep time by weeks  
- Reduce reliance on manual spreadsheets and consultants  
- Stay continuously compliant as their product and team evolve  

If SOC 2 is on your roadmap (or already a pain point), I’d be happy to show you how ComplAI works in a short, no‑pressure demo.

Would it make sense to explore this for {{Company Name}}?

Best regards,  
{{Your Name}}  
{{Title}} | ComplAI  
{{E

In [10]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-5.2-chat-latest"
)


In [11]:
message = "Write a cold sales email"

with trace("Pick the best cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )
    outputs = [result.final_output for result in results]
    emails = "\n\n".join(f"Option {i+1}:\n{output}" for i, output in enumerate(outputs))
    picker_input = f"Here are three cold sales email options:\n\n{emails}\n\nPick the best one."
    picker_result = await Runner.run(sales_picker, input=picker_input)
    print("\n\n--- Selected Best Cold Email ---\n")
    print(picker_result.final_output)



--- Selected Best Cold Email ---

Subject: Cut SOC 2 prep time by 50% with AI

Hi {{First Name}},

Preparing for SOC 2 can be painful—manual evidence collection, endless checklists, and last‑minute audit stress.

ComplAI automates SOC 2 readiness with AI:
- Auto‑collects and maps evidence  
- Flags gaps before auditors do  
- Keeps you continuously compliant, not just audit‑ready

Teams typically cut prep time by 50%+ and pass audits faster.

Worth a quick 15‑minute chat to see if this fits {{Company Name}}?

Best,  
{{Your Name}}  
ComplAI


In [12]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-5.2-chat-latest"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-5.2-chat-latest"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-5.2-chat-latest"
)

In [13]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoffs=[], model='gpt-5.2-chat-latest', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [14]:
@function_tool
def send_email_tool(subject: str, body: str) -> Dict[str, str]:
    """
    Sends an email using SendGrid.
    Args:
        subject (str): The subject of the email.
        body (str): The body content of the email.
    Returns:
        Dict[str, str]: A dictionary with the status of the email sending operation.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY')) 
    from_email = Email("muhammadhassan.ai@muhammadhassan.tech")  
    to_email = To("muhammad.hassan.519570@gmail.com") 
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": f"Email sent with status code {response.status_code}"}

In [18]:
send_email_tool

FunctionTool(name='send_email_tool', description='Sends an email using SendGrid.\nArgs:\n    subject (str): The subject of the email.\n    body (str): The body content of the email.\nReturns:\n    Dict[str, str]: A dictionary with the status of the email sending operation.', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'body': {'title': 'Body', 'type': 'string'}}, 'required': ['subject', 'body'], 'title': 'send_email_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7b66537b44a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [20]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email_tool]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7b6636c65760>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7b6636c64720>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='

In [21]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-5.2-chat-latest")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)